In [ ]:
import yaml
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from SimBMVtool import SimBMVtool as sim

path_config = './config_real.yaml'

# Example dataset

You will need to copy the LST Analysis School data, preferably without the index tables (new ones will be created with the correct paths)

`/fefs/aswg/workspace/analysis-school-2024/DL3/Crab/dl3_*.fits`

Use the destination path as input in the configuration file `config_real.yaml`


In [ ]:
accmodel = sim.BaseSimBMVCreator()
accmodel.init_config(path_config)
accmodel.plot_exclusion_mask()

do_accmodel = False
if do_accmodel: accmodel.do_acceptance_modelisation()
else: 
    accmodel.load_observation_collection(from_index=True)
    accmodel.load_output_background_irfs()

In [ ]:
# Plot the model obtained for the 4th run
accmodel.plot_model(data='acceptance',irf='output', i_irf=3)

In [ ]:
# Create a zenith binned collection of models, with 4 bins by default
accmodel.plot_zenith_binned_model(data='acceptance',irf='output', zenith_bins='auto')

In [ ]:
accmodel.create_zenith_binned_collections(collections=['observation'],zenith_bins='auto')
for coszd_bin in accmodel.zenith_binned_obs_collection.keys():
    print(f"zd = {np.rad2deg(np.acos(coszd_bin)):.1f}°: ", accmodel.zenith_binned_obs_collection[coszd_bin].ids)

In [ ]:
# Plot only one zenith bin
accmodel.plot_zenith_binned_model(data='acceptance',irf='output', i_bin=0)

In [ ]:
e_min, e_max = (accmodel.e_min, 1*u.TeV)
offset_max_dataset, offset_max_map = (2.5, 2.5) # offset_max_dataset >= offset_max_map
accmodel.axis_info_dataset = [e_min, e_max, offset_max_dataset * u.deg, accmodel.nbin_offset_irf]
accmodel.axis_info_map = [e_min, e_max, offset_max_map * u.deg, accmodel.nbin_offset_irf]
accmodel.plot_skymaps('ring')

In [ ]:
# Plotting everything takes some time, and you may need to look only at one map
stacked_dataset = accmodel.get_stacked_dataset(bkg_method='ring', axis_info=accmodel.axis_info_dataset)
skymaps = sim.get_skymaps_dict(stacked_dataset, accmodel.exclude_regions, accmodel.correlation_radius, accmodel.correlate_off, 'all')
print(skymaps)

sim.plot_skymap_from_dict(skymaps, 'counts')

# Crop the map and/or reduce the size of the plot
sim.plot_skymap_from_dict(skymaps, 'excess', crop_width=1*u.deg, figsize=(4,4))